In [ ]:
import io
import json
import pandas as pd
from google.colab import files

uploaded = files.upload()

Saving intents.json to intents.json


In [ ]:
df = pd.read_json(io.StringIO(uploaded.get('intents.json').decode('utf-8')))

In [ ]:
df

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'How ar..."
1,"{'tag': 'goodbye', 'patterns': ['cya', 'see yo..."
2,"{'tag': 'creator', 'patterns': ['what is the n..."
3,"{'tag': 'name', 'patterns': ['name', 'your nam..."
4,"{'tag': 'hours', 'patterns': ['timing of colle..."
5,"{'tag': 'number', 'patterns': ['more info', 'c..."
6,"{'tag': 'course', 'patterns': ['list of course..."
7,"{'tag': 'fees', 'patterns': ['information abou..."
8,"{'tag': 'location', 'patterns': ['where is the..."
9,"{'tag': 'hostel', 'patterns': ['hostel facilit..."


In [ ]:
import nltk
import pickle
import random

import numpy as np
import tensorflow as tf

from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
word = []
classes = []
document = []
ignore = ['.',',','!', '?']

for intents in df['intents']:
  for pattern in intents['patterns']:
    wordList = nltk.word_tokenize(pattern)
    word.extend(wordList)
    document.append((wordList, intents['tag']))
    if intents['tag'] not in classes:
      classes.append(intents['tag'])

words = [lemmatizer.lemmatize(word) for word in word if word not in ignore]

words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [ ]:
training = []
output = [0] * len(classes)

for doc in document:
    bag = []
    wordPatterns = doc[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(output)
    outputRow[classes.index(doc[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print('Done')

Epoch 1/200
81/81 [==============================] - 1s 2ms/step - loss: 3.5771 - accuracy: 0.0617
Epoch 2/200
81/81 [==============================] - 0s 2ms/step - loss: 3.3879 - accuracy: 0.1284
Epoch 3/200
81/81 [==============================] - 0s 2ms/step - loss: 3.1213 - accuracy: 0.1877
Epoch 4/200
81/81 [==============================] - 0s 3ms/step - loss: 2.7843 - accuracy: 0.2741
Epoch 5/200
81/81 [==============================] - 0s 2ms/step - loss: 2.5635 - accuracy: 0.3358
Epoch 6/200
81/81 [==============================] - 0s 2ms/step - loss: 2.3692 - accuracy: 0.3827
Epoch 7/200
81/81 [==============================] - 0s 2ms/step - loss: 2.2034 - accuracy: 0.4148
Epoch 8/200
81/81 [==============================] - 0s 2ms/step - loss: 2.0358 - accuracy: 0.4321
Epoch 9/200
81/81 [==============================] - 0s 2ms/step - loss: 1.8872 - accuracy: 0.4963
Epoch 10/200
81/81 [==============================] - 0s 2ms/step - loss: 1.6014 - accuracy: 0.5580
Epoch 11/

In [ ]:
from nltk.stem import WordNetLemmatizer
from keras.models import load_model

lemmatizer = WordNetLemmatizer()

In [ ]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words (sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class (sentence):
    bow = bag_of_words (sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes [r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice (i['responses'])
            break
    return result

print("GO! Bot is running!")

GO! Bot is running!


In [ ]:
while True:
    message = input("ME: ")
    if(message == 'Bye'):
      ints = predict_class(message)
      res = get_response(ints, df)
      print(res)
      break;
    ints = predict_class (message)
    res = get_response (ints, df)
    print (res)